In [1]:
import spacy
import csv
import pandas as pd

from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER
from spacy.lang.char_classes import CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex

In [2]:
nlp = spacy.load("en_core_web_sm")

# modify tokenizer infix patterns for avoiding hyphenated words being split (several terms are hyphenated)

infixes = (
    LIST_ELLIPSES
    + LIST_ICONS
    + [
        r"(?<=[0-9])[+\-\*^](?=[0-9-])",
        r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
            al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
        ),
        r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
        r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
    ]
)

infix_re = compile_infix_regex(infixes)
nlp.tokenizer.infix_finditer = infix_re.finditer

In [3]:
def read_tsv(link):
    with open(link, "r", encoding="utf-8") as tsvfile:
        reader = csv.reader(tsvfile, delimiter="\t")
        next(reader) # skip header
        return {row[2].split(": ")[1] for row in reader}

terms_lexicon = read_tsv("../swimming-terms.tsv") 
# list of extracted terms

In [59]:
terms_lexicon

{'physique',
 'bachman',
 'pb improvement',
 'term of volume and intensity',
 'young',
 'firma',
 'meter swimming',
 'speed swimming',
 'algorithm',
 'root-\xad‐mean-\xad‐square',
 'potential pb',
 'lung',
 'share',
 'catch point',
 'beginning of backstroker',
 'biomechanists',
 'breaststroke swimming',
 'center of mass velocity',
 'implication of current fina',
 'ernest',
 'regularity',
 'combination of right hand',
 'type of stroke',
 'model lsb302',
 'mental fortitude',
 'repeated sprint training',
 'negative factor',
 'huge increase',
 'palmer',
 'post-\xad‐test',
 'factor',
 'total number of stroke',
 'swimmer arm',
 'line with body',
 'morphological change',
 'bicycle short-\xad‐sprint training',
 'central training effect',
 'production of electrode',
 'ergogenic aid',
 'additional oxygen',
 'scapular stabilization',
 'large effect',
 'dave tanner1',
 'result of propulsion',
 'average',
 'relevance of current datum',
 'joint torques',
 'muscle imbalance',
 'loftice',
 'olympic co

In [5]:
# open 1 document, spacy it
with open("../corpus_txt/manuscript-maglischo-vol22.txt") as f: # corpus here
    doc = f.readlines()[0]
    doc = nlp(doc)


In [18]:
doc[0].lemma_

'Swimming'

In [29]:
df = pd.DataFrame([item for sub in [[tok.text for tok in s]+["<eos>"] for s in doc.sents] for item in sub], columns=["token"])

df["lemma"] = [item for sub in [[token.lemma_.lower() if token.lemma_ != '-PRON-' else token.lower_ for token in s]+["<eos>"] for s in doc.sents] for item in sub]


In [30]:
df

,token,lemma
0,Swimming,swimming
1,Research,research
2,",",","
3,Vol,vol
4,.,.
...,...,...
9869,Swimming,swimming
9870,.,.
9871,<eos>,<eos>
9872,23,23


In [31]:
# populate 3rd column with "O" tags
df["tag"] = ["O"]*len(df)

In [35]:
max_term_len = max([len(x.split(" ")) for x in terms_lexicon])
print(max_term_len)

6


# Attempts

## 2 df n grams

In [114]:
# Creating a new df which contains the terms ngrams (n from 1 to 6)
# And the index of the first word of this ngram in the original df

ngram_df_entries = []

for i in range(1,max_term_len+1):
    for j in range(len(df)):
        if j+i > len(df):
            pass
        n_gram = ' '.join(df.iloc[j: j+i]["lemma"])
        if n_gram in terms_lexicon:
            ngram_df_entries.append([j, n_gram, i])

In [115]:
ngram_df = pd.DataFrame(columns=["starting_index", "ngram", "ngram_len"], data=ngram_df_entries)

In [116]:
ngram_df

,starting_index,ngram,ngram_len
0,0,swimming,1
1,14,time,1
2,21,swimming,1
3,22,backstroke,1
4,25,ernest,1
...,...,...,...
3180,2958,middle and right hand photo,5
3181,7438,period of trial and error,5
3182,7901,short and long backstroke event,5
3183,7919,male and female backstroke swimmer,5


In [130]:
# Filtering duplicates (keeping the biggest ngrams)
ngram_df["is_duplicate"] = ngram_df.duplicated(subset="starting_index", keep='last')

In [132]:
ngram_df = ngram_df.query('is_duplicate == False')

In [137]:
for index, row in ngram_df.iterrows():
    #breakpoint()
    if row.ngram_len == 1:
        df.iloc[row.starting_index].tag = "I"
    else:
        for i in range(row.ngram_len):
            if i==0:
                tag = "B"
            else:
                tag = "I"
            df.iloc[row.starting_index+i].tag = tag

## exploration

In [138]:
df.tag.value_counts()

O    6981
I    2295
B     598
Name: tag, dtype: int64

In [139]:
df.query('tag=="B"')

,token,lemma,tag
0,Swimming,swimming,B
29,Lazy,lazy,B
49,best,good,B
65,unique,unique,B
90,unique,unique,B
...,...,...,...
9790,stroke,stroke,B
9798,elite,elite,B
9834,Human,human,B
9852,Online,online,B


In [140]:
"swimming" in terms_lexicon

True

In [141]:
df.to_csv("dataset.txt", sep=" ",header=False, index=False, columns=["token", "tag"])

In [142]:
df

,token,lemma,tag
0,Swimming,swimming,B
1,Research,research,I
2,",",",",O
3,Vol,vol,O
4,.,.,O
...,...,...,...
9869,Swimming,swimming,I
9870,.,.,O
9871,<eos>,<eos>,O
9872,23,23,O
